In [80]:
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn.functional as F
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import transforms
from pathlib import Path

In [81]:
def unzip_dataset(INPATH, OUTPATH):
    with zipfile.ZipFile(INPATH) as zf:
        zf.extractall(OUTPATH)
    
unzip_dataset(INPATH = './train_data.zip', OUTPATH = './')
unzip_dataset(INPATH = './test_data.zip', OUTPATH = './')

In [82]:
train_dir = '../Casting_products_pytorch/train_data/'
test_dir = '../Casting_products_pytorch/test_data/'
train_df = pd.read_csv('../Casting_products_pytorch/train.csv')

In [83]:
train_df, val_df = train_test_split(np.array(train_df), test_size=0.1)

In [84]:
transform = torchvision.transforms.ToTensor()

In [85]:
class Casting_products_Datasets(Dataset):
    def __init__(self, dir_name, csv_file, transform=None):
        self.dir_name = dir_name
        self.csv_file = csv_file
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self, index):
        file = self.csv_file[index][0]
        label = np.array([self.csv_file[index][1]], np.float32)
        image = Image.open(self.dir_name + file)
        image = self.transform(image)

        return image, label

In [86]:
train_dataset = Casting_products_Datasets(train_dir, train_df, transform)
val_dataset = Casting_products_Datasets(train_dir, val_df, transform)

In [87]:
batch_size = 5

In [88]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [89]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=12)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=24)
        self.fc1 = nn.Linear(24*144*144, 128)
        self.fc2 = nn.Linear(128, 48)
        self.fc3 = nn.Linear(48, 1)
    
    def forward(self, x):
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = x.view(-1, 24*144*144)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
model = Network()

In [90]:
loss_fnc = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters())
record_loss_train = []
record_loss_val = []

In [91]:
images, labels = iter(val_dataloader).next()

In [93]:
images.size()

torch.Size([5, 3, 300, 300])

In [98]:
for i in range(20):
    model.train()
    loss_train = 0
    for j, (x, t) in enumerate(train_dataloader):
        y = model(x)
        loss = loss_fnc(y, t)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train /= j+1
    record_loss_train.append(loss_train)

    model.eval()
    y_test = model(images)
    loss_test = loss_fnc(y_test, labels).item()
    record_loss_val.append(loss_test)

    if i%1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)


Epoch: 0 Loss_Train: 3.022941598609224 Loss_Test: 0.49720340967178345
Epoch: 1 Loss_Train: 0.8459209305447681 Loss_Test: 2.2636728286743164
Epoch: 2 Loss_Train: 0.6083684311726958 Loss_Test: 3.7307732105255127
Epoch: 3 Loss_Train: 0.5952305339794192 Loss_Test: 1.7784528732299805
Epoch: 4 Loss_Train: 0.39146597505443625 Loss_Test: 0.42805081605911255
Epoch: 5 Loss_Train: 0.4656173306491433 Loss_Test: 0.8876969218254089
Epoch: 6 Loss_Train: 0.3794706905170137 Loss_Test: 3.2638230323791504


KeyboardInterrupt: 